## Finite State Projection Algorithm
**Stochastic Kinetics of mRNA Molecules in a General Transcription Model**

*Yuntao Lu and Yunxin Zhang*

School of Mathematical Sciences, Fudan University, Shanghai 200433, China

Email: `yuntaolu22@m.fudan.edu.cn` and `xyz@fudan.edu.cn`

This script is written to time FSP, generating date used to plot Figure 4. 

This script implements original Finite State Projection (FSP) algorithm (proposed in
`Munsky, B., and M. Khammash, 2006. The Finite State Projection Algorithm for the Solution of the Chemical Master Equation. J. Chem. Phys. 124:044104.`) to study the general transcription model in our paper. The main objective is to time the running times of FSP. The results are plotted in Figure 4 in our paper.

Note that we only time one final execution of `expm_multiply(A * t , p0)`, which calculates the product $\exp{(tA)}p(0)$. Matrix $A$, or equivalently the subset of states, is determined such that the truncation error, defined as $1-\pmb{1}^T\exp{(tA)}p(0)$, is below the prescribed tolerance $\epsilon=1e-5$. Therefore, the preceding iterative process of expanding the state subset (or equivalently the matrix $A$) to achieve this error bound is excluded from the timing. 

In [1]:
import numpy as np
from scipy.sparse import bmat
from scipy.sparse.linalg import expm_multiply
import time

In [2]:
def construct_block_matrix(D0, D1, delta=1, K=10):
    """
    The matrix has (K+1) block rows and columns, resulting in a total size of (n*(K+1)) x (n*(K+1)).

    The structure of the block matrix is as follows:

        [ D0^T           delta I                   ]
        [ D1^T   D0^T - delta I    2*delta I         ]
        [        D1^T     D0^T - 2*delta I   ...     ]
        [               ...             ...         ]
        [                         D1^T  D0^T - K*delta I ]

    That is:
    - The main diagonal blocks are: D0^T - k * delta * I for row k (k = 0 to K)
    - The super-diagonal (upper diagonal) blocks are: (k+1) * delta * I for row k
    - The sub-diagonal (lower diagonal) blocks are: D1^T (constant across all applicable rows)

    Parameters:
    -----------
    D0 : numpy.ndarray, shape (n, n). A square matrix. 
    D1 : numpy.ndarray, shape (n, n). A square matrix.
    delta : float, optional (default=1)
    K : int, optional (default=10)
        Determines the number of block rows/columns minus one. The total number of 
        block rows and columns is (K + 1), so K=10 results in 11 block rows.

    Returns:
    --------
    scipy.sparse matrix, shape (n*(K+1), n*(K+1))
        The constructed sparse block matrix in Compressed Sparse Row ('csr') format.
        This format is efficient for arithmetic operations and matrix-vector products.

    Raises:
    -------
    ValueError:
        If D0 or D1 are not square, or if they do not have the same dimensions.

    Notes:
    ------
    - The function transposes D0 and D1 once at the beginning to avoid repeated transposition.
    - Uses scipy.sparse.bmat to efficiently assemble the block structure.
    """
    # Get the dimension of the input square matrices
    n = D0.shape[0]

    # Validate input dimensions
    if D0.shape != (n, n) or D1.shape != (n, n):
        raise ValueError("D0 and D1 must be square matrices of the same size (n x n)")

    # Compute the transposes once to avoid repeated computation
    D0T = D0.T  # Transpose of D0
    D1T = D1.T  # Transpose of D1

    # List to store each row of blocks (each row is a list of blocks)
    blocks = []

    # Iterate over each block row index k (from 0 to K inclusive)
    for k in range(K + 1):
        # Initialize a list for the current row of blocks, with None placeholders
        row_blocks = [None] * (K + 1)

        # --- Main diagonal block ---
        # At position (k, k): D0^T - k * delta * Identity matrix
        main_block = D0T - k * delta * np.eye(n)
        row_blocks[k] = main_block

        # --- Super-diagonal (upper diagonal) block ---
        # At position (k, k+1): (k+1) * delta * Identity matrix
        # Only add if we are not in the last row (k < K)
        if k < K:
            upper_block = (k + 1) * delta * np.eye(n)
            row_blocks[k + 1] = upper_block

        # --- Sub-diagonal (lower diagonal) block ---
        # At position (k, k-1): D1^T
        # Only add if we are not in the first row (k > 0)
        if k > 0:
            row_blocks[k - 1] = D1T

        # Append the current row of blocks to the list of all block rows
        blocks.append(row_blocks)

    # Assemble the full block matrix using scipy.sparse.bmat
    # Format 'csr' (Compressed Sparse Row) is chosen for efficient arithmetic and multiplication
    result = bmat(blocks, format='csr')

    return result

In [3]:
# import parameter matrices prepared for figures in the paper
import Parameters_for_Figures

In [4]:
delta=1

In [5]:
def FSP_timing(D0, D1, delta=1, T=10, tol=1e-5):
    # Number of block levels (K in the matrix construction), resulting in (Block + 1) block rows/columns
    Block = 10
    
    # Get the size N of the original square matrices D0 and D1 (assumed to be N x N)
    N = D0.shape[0]
    
    # Construct the large block matrix A using the custom function
    A = construct_block_matrix(D0, D1, delta, K=Block)
    
    # Convert the sparse block matrix A to a dense NumPy array.
    # Note: This step can be very memory-intensive for large N or Block.
    # Only feasible if N*(Block+1) is not too large (e.g., < few thousand).
    A = A.toarray()
    
    # Total evolution time. We approximate the probability distribution at time t=T.
    # T = 10
    
    # Initialize the initial probability vector p0.
    # Size: N * (Block + 1) — one entry for each state in each block level.
    p0 = np.zeros(N * (Block + 1))
    
    # Set initial condition as a Dirac distribution.
    # This assumes that the system begins with no mRNA molecules and the gene being in state 1.
    p0[0] = 1
    
    # Compute the action of the matrix exponential exp(A * T) on the vector p0.
    # - Uses expm_multiply for efficiency and numerical stability.
    # - Avoids explicitly computing the full matrix exponential exp(A*T), which is expensive.
    # - Returns pT = exp(A*T) @ p0, representing the truncated probability density function at time t=T.
    pT = expm_multiply(A * T, p0)
    
    # Compute the sum of all entries in this probability density function vector.
    # For a well-approximated probability distribution, this value should be very close to 1.0.
    total = np.sum(pT)
    
    # Print the total probability (should be ~1.0 if probability is conserved)
    # print(f" Current Block Number is {Block}; sum of probability is {total}")
    
    
    
    while total < 1-tol:
        Block+=10
      
        A = construct_block_matrix(D0, D1, delta, K=Block)
        A=A.toarray()
        
        p0 = np.zeros(N * (Block + 1))
        p0[0] = 1 
    
        # pT is a ndarray of size (N * (Block + 1) , 1)
    
        pT = expm_multiply(A * T, p0)
    
        total=np.sum(pT)
        
        # print(f" Current Block Number is {Block}; sum of probability is {total}")
    
    print(f"The tolerance is set as {tol}; Converged with Block Number={Block}; sum of probability is={total}")
    # Timing the computation of expm_multiply step
    start_time = time.time()
    
    pT = expm_multiply(A * T, p0)
    
    # Timing ends here
    end_time = time.time()
    timing = end_time - start_time
    
    print(f"Computation time of `expm_multiply` step is : {timing:.4f} seconds.")
    return timing, pT, total, Block
    

In [6]:
times=[]
for i in range(1,21):
    print(f'Model of order {i}: Starting')
    time1=FSP_timing(Parameters_for_Figures.generate_D0n(i),
                     Parameters_for_Figures.generate_D1n(i),
                     delta=1, T=10, tol=1e-5)[0]
    time2=FSP_timing(Parameters_for_Figures.generate_D0n(i),
                     Parameters_for_Figures.generate_D1n(i),
                     delta=1, T=10, tol=1e-5)[0]
    time3=FSP_timing(Parameters_for_Figures.generate_D0n(i),
                     Parameters_for_Figures.generate_D1n(i),
                     delta=1, T=10, tol=1e-5)[0]
    times.append(float(np.average([time1,time2,time3])))
    print(f'Model of order {i}: Completed!')

Model of order 1: Starting
The tolerance is set as 1e-05; Converged with Block Number=10; sum of probability is=0.9999993448863428
Computation time of `expm_multiply` step is : 0.0186 seconds.
The tolerance is set as 1e-05; Converged with Block Number=10; sum of probability is=0.9999993448863428
Computation time of `expm_multiply` step is : 0.0119 seconds.
The tolerance is set as 1e-05; Converged with Block Number=10; sum of probability is=0.9999993448863428
Computation time of `expm_multiply` step is : 0.0092 seconds.
Model of order 1: Completed!
Model of order 2: Starting
The tolerance is set as 1e-05; Converged with Block Number=20; sum of probability is=0.9999999994162604
Computation time of `expm_multiply` step is : 0.0144 seconds.
The tolerance is set as 1e-05; Converged with Block Number=20; sum of probability is=0.9999999994162604
Computation time of `expm_multiply` step is : 0.0143 seconds.
The tolerance is set as 1e-05; Converged with Block Number=20; sum of probability is=0.

In [7]:
# Save the results in a `.npy` file
# np.save('FSP_times.npy', times)

In [8]:
print(times)

[0.013245423634847006, 0.01446827252705892, 0.025301059087117512, 0.03003231684366862, 0.05527663230895996, 0.10126868883768718, 0.2418358325958252, 0.22104652722676596, 0.6476337909698486, 0.19591140747070312, 0.27911965052286786, 0.4375251928965251, 0.4666256109873454, 0.8053211371103922, 1.4011401335398357, 2.276273012161255, 3.0701301097869873, 5.218783299128215, 6.45630145072937, 8.783692439397177]


In [9]:
# load .npy file
# loaded_array = np.load('FSP_times.npy')
# loaded_list_from_array = loaded_array.tolist()
# print(loaded_list_from_array)

[0.013245423634847006, 0.01446827252705892, 0.025301059087117512, 0.03003231684366862, 0.05527663230895996, 0.10126868883768718, 0.2418358325958252, 0.22104652722676596, 0.6476337909698486, 0.19591140747070312, 0.27911965052286786, 0.4375251928965251, 0.4666256109873454, 0.8053211371103922, 1.4011401335398357, 2.276273012161255, 3.0701301097869873, 5.218783299128215, 6.45630145072937, 8.783692439397177]
